In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
demand = np.array([0,1,2,3,4,5])
frequency = np.array([15,30,60,120,45,30])
demand_probs = frequency/np.sum(frequency)
demand_cum_probs = np.cumsum(demand_probs)
lead_time = np.array([1,2,3])
frequency = np.array([10,25,15])
lead_time_probs = frequency/np.sum(frequency)
lead_time_cum_probs = np.cumsum(lead_time_probs)

In [14]:
m = 1000
N = 100 
daily_cost_values = np.zeros((m,N))
for i in range(N):
    Q = 16
    R = 7
    I = 10
    EI = []
    LS = []
    lead_time_new = 0
    order = False
    daily_cost = []
    for j in range(m):
        if (order==True):
            lead_time_new = lead_time_new - 1
            if (lead_time_new==0):
                I = I + Q
                order= False
        demand_rv = np.random.uniform(0,1,1)
        index_value = min(np.argwhere(demand_cum_probs>demand_rv).ravel())
        demand_realized = demand[index_value]
        EI.append(max(0,I-demand_realized))
        LS.append(max(0,demand_realized-I))
        mismatch_cost = 0.5*EI[-1]+8*LS[-1]
        ordering_cost = 0
        I = EI[-1]
        if (I<R and order==False):
            lead_time_rv = np.random.uniform(0,1,1)
            index_value = min(np.argwhere(lead_time_cum_probs>lead_time_rv).ravel())
            lead_time_realized = lead_time[index_value]
            lead_time_new = lead_time_realized
            order=True
            ordering_cost = 10
        daily_cost.append(mismatch_cost+ordering_cost)
    daily_cost_values[:,i]=daily_cost

# calculate the mean of each sample
mean_sample = np.mean(daily_cost_values, axis = 0)
# calculate standard deviation
sigma = np.std(mean_sample)
# Mean of means
f_bar_bar = np.mean(mean_sample)
# Lower bound of CI
lower_bound = f_bar_bar-1.96*sigma/np.sqrt(N)
# Upper bound of CI
upper_bound = f_bar_bar+1.96*sigma/np.sqrt(N)
# Summary statistics Lower Bound, Mean of Means, Upper Bound
list[lower_bound,f_bar_bar,upper_bound]

list[6.676079091814609, 6.692345, 6.708610908185392]

In [20]:
def simulate_inventory_cost(Q_values, R_values, m=1000, N=100, demand=[], demand_cum_probs=[], lead_time=[], lead_time_cum_probs=[]):
    """
    Simulates inventory costs for a range of Q and R values and finds the optimal combination
    resulting in the lowest summary statistic (mean of means).

    Parameters:
    - Q_values: List of possible order quantities (Q) to test.
    - R_values: List of possible reorder points (R) to test.
    - m: Number of days in a single simulation (default: 1000).
    - N: Number of simulation runs for each Q and R combination (default: 100).
    - demand: List of demand values.
    - demand_cum_probs: Cumulative probabilities corresponding to demand values.
    - lead_time: List of lead times.
    - lead_time_cum_probs: Cumulative probabilities corresponding to lead times.

    Returns:
    - results: List of dictionaries with Q, R, mean of means, lower bound, and upper bound.
    - optimal: Dictionary with the optimal Q, R, and summary statistics.
    """
    results = []

    for Q in Q_values:
        for R in R_values:
            daily_cost_values = np.zeros((m, N))
            
            for i in range(N):
                I = 10  # Initial inventory level
                lead_time_new = 0
                order = False
                daily_cost = []

                for j in range(m):
                    if order:
                        lead_time_new -= 1
                        if lead_time_new == 0:
                            I += Q
                            order = False
                    
                    demand_rv = np.random.uniform(0, 1, 1)
                    index_value = min(np.argwhere(demand_cum_probs > demand_rv).ravel())
                    demand_realized = demand[index_value]
                    
                    EI = max(0, I - demand_realized)
                    LS = max(0, demand_realized - I)
                    mismatch_cost = 0.5 * EI + 8 * LS
                    ordering_cost = 0
                    
                    I = EI
                    
                    if I < R and not order:
                        lead_time_rv = np.random.uniform(0, 1, 1)
                        index_value = min(np.argwhere(lead_time_cum_probs > lead_time_rv).ravel())
                        lead_time_realized = lead_time[index_value]
                        lead_time_new = lead_time_realized
                        order = True
                        ordering_cost = 10
                    
                    daily_cost.append(mismatch_cost + ordering_cost)
                
                daily_cost_values[:, i] = daily_cost

            # Calculate summary statistics
            mean_sample = np.mean(daily_cost_values, axis=0)
            sigma = np.std(mean_sample)
            f_bar_bar = np.mean(mean_sample)
            lower_bound = f_bar_bar - 1.96 * sigma / np.sqrt(N)
            upper_bound = f_bar_bar + 1.96 * sigma / np.sqrt(N)

            results.append({
                "Q": Q,
                "R": R,
                "Mean of Means": f_bar_bar,
                "Lower Bound": lower_bound,
                "Upper Bound": upper_bound
            })

    # Find the optimal Q and R combination
    optimal = min(results, key=lambda x: x["Mean of Means"])

    return results, optimal


# Example of usage (requires demand, demand_cum_probs, lead_time, lead_time_cum_probs defined)
# Q_values = [10, 20, 30]
# R_values = [5, 10, 15]
# results, optimal = simulate_inventory_cost(Q_values, R_values, demand=demand, 
#                                            demand_cum_probs=demand_cum_probs, 
#                                            lead_time=lead_time, 
#                                            lead_time_cum_probs=lead_time_cum_probs)

# Results and optimal configuration are stored in `results` and `optimal` respectively.



In [ ]:
# Generate ranges for Q and R
Q_values = list(range(5, 26, 1))  # Q values from 5 to 25
R_values = list(range(5, 26, 1))  # R values from 5 to 25

# Run the simulation
results, optimal = simulate_inventory_cost(Q_values, R_values, m=1000, N=100,
                                           demand=demand, 
                                           demand_cum_probs=demand_cum_probs, 
                                           lead_time=lead_time, 
                                           lead_time_cum_probs=lead_time_cum_probs)

# Print results
print("All Results:")
for result in results:
    print(result)

# Print optimal configuration
print("\nOptimal Configuration:")
print(optimal)

All Results:
{'Q': 5, 'R': 5, 'Mean of Means': 10.200849999999999, 'Lower Bound': 10.006168810440247, 'Upper Bound': 10.395531189559751}
{'Q': 5, 'R': 6, 'Mean of Means': 10.003400000000001, 'Lower Bound': 9.866554110977349, 'Upper Bound': 10.140245889022653}
{'Q': 5, 'R': 7, 'Mean of Means': 9.906, 'Lower Bound': 9.8570676667223, 'Upper Bound': 9.954932333277702}
{'Q': 5, 'R': 8, 'Mean of Means': 9.6262, 'Lower Bound': 9.506290426130356, 'Upper Bound': 9.746109573869646}
{'Q': 5, 'R': 9, 'Mean of Means': 9.52615, 'Lower Bound': 9.439735219271238, 'Upper Bound': 9.61256478072876}
{'Q': 5, 'R': 10, 'Mean of Means': 9.6448, 'Lower Bound': 9.502370075694747, 'Upper Bound': 9.787229924305253}
{'Q': 5, 'R': 11, 'Mean of Means': 9.630799999999999, 'Lower Bound': 9.447109138796726, 'Upper Bound': 9.814490861203272}
{'Q': 5, 'R': 12, 'Mean of Means': 9.627399999999998, 'Lower Bound': 9.514330268743574, 'Upper Bound': 9.740469731256422}
{'Q': 5, 'R': 13, 'Mean of Means': 9.614550000000001, 'Low

In [ ]:
print